In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

# 设置GPU环境
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定义CNN+RNN模型
class CNNtoRNN(nn.Module):
    def __init__(self):
        super(CNNtoRNN, self).__init__()
        # CNN层
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        # RNN层
        self.rnn = nn.RNN(64 * 7, 128, 2, batch_first=True)
        self.fc = nn.Linear(128, 10)

    def forward(self, x):
        # CNN部分
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # 调整形状以适配RNN输入
        x = x.permute(0, 2, 1, 3).contiguous()
        x = x.view(x.size(0), x.size(1), -1)
        # RNN部分
        h0 = torch.zeros(2, x.size(0), 128).to(device)
        x, _ = self.rnn(x, h0)
        x = x[:, -1, :]
        # 全连接层
        x = self.fc(x)
        return x

# 数据加载和预处理
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

# 创建CNN+RNN模型，并将其移至GPU
model = CNNtoRNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 1
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
        images = images.to(device)
        labels = labels.to(device)

        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(trainloader)}], Loss: {loss.item():.4f}')

# 测试模型
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Epoch [1/1], Step [100/938], Loss: 0.1844
Epoch [1/1], Step [200/938], Loss: 0.0510
Epoch [1/1], Step [300/938], Loss: 0.1394
Epoch [1/1], Step [400/938], Loss: 0.1246
Epoch [1/1], Step [500/938], Loss: 0.1338
Epoch [1/1], Step [600/938], Loss: 0.0528
Epoch [1/1], Step [700/938], Loss: 0.0146
Epoch [1/1], Step [800/938], Loss: 0.0723
Epoch [1/1], Step [900/938], Loss: 0.0863
Test Accuracy of the model on the 10000 test images: 98.05 %
